In [ ]:
import sys
import pandas as pd
import numpy as np

sys.path.append('.')
data_daily = pd.read_csv('./data/48_Industry_Portfolios_Daily.csv', index_col=0)

# Trim the data
data_daily = data_daily.iloc[('197401' <= data_daily.index.values.astype(str)) & (data_daily.index.values.astype(str) <= '202012')]

# Convert the index to datetime
data_daily.index = pd.to_datetime(data_daily.index, format='%Y%m%d')
daily_date = data_daily.index.date

# The rate of return matrix, fill the missing values with 100%
Return = data_daily.values.T.astype(float)
Return[Return < -99]
Gross_return = (Return + 100) / 100 # Gross Return

print(Gross_return.shape)

# Check the first few rows of the dataset
data_daily.head()

window_size = 120

In [ ]:
from ucb import UCB
ucb = UCB(Gross_return, window_size)
ucb.run()
#ucb.test()
ucb_return = ucb.get_return()

In [ ]:
from psr import PSR
psr = PSR(Gross_return, window_size)
psr.run()

In [ ]:
# Baseline
constant_weight_rebalance = np.cumprod(Gross_return[:, window_size:].mean(axis=0))
equal_weight_portfolio = np.mean(np.cumprod(Gross_return[:, window_size:], axis=1), axis=0)